In [12]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler, MinMaxScaler

import lightgbm as lgb
import catboost as cb
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor

from tsfresh.examples import load_robot_execution_failures
from tsfresh import extract_features, select_features
import optuna

from common import EP
from dfdb import DFDB

import types
import copy

In [2]:
%%HTML
<style>
   div#notebook-container    { width: 95%; }
   div#menubar-container     { width: 65%; }
   div#maintoolbar-container { width: 99%; }
</style>

In [3]:
pd.set_option('display.max_rows', 2000)
pd.set_option('display.max_colwidth', -1)

In [4]:
df_train = pd.read_pickle('../feats/df_train.pkl')
df_test = pd.read_pickle('../feats/df_test.pkl')

In [5]:
df_train['label'] = df_train['y'].apply(lambda x:  int(x) if x<15 else 15)
group = df_train['season'].values
group[np.where(group==17)[0]] = 1
df_train['group'] = group
df_train = df_train.drop(columns=['season'])

In [6]:
test_X = df_train.drop(columns=['y','index','group','label']).copy()
test_X.index = df_train['index']
test_y = df_train['y'].copy()
test_y.index = df_train['index']
tsfresh_columns = select_features(test_X, test_y).columns.tolist()

In [7]:
original_columns = df_train.columns.drop(['index','y','label','group']).tolist()

In [8]:
db = DFDB('../trial/decisiontrees.pkl', auto_commit=False)

In [9]:
catboost_columns = ['spkt_welch_density__coeff_3',
 'spkt_welch_densitycoeff_2',
 'abs_q25_5',
 'abs_q75_6',
 'q05_roll_std_1000',
 'abs_q75_7',
 'abs_q95_2',
 'q05_5',
 'abs_q75_2',
 '5000skewness_max_',
 'fft_coefficientcoeff_80__attr_"imag"',
 "change_quantiles{'ql': 0.2, 'qh': 0.8, 'isabs': False, 'f_agg': 'var'}",
 '5000kurtosis_mean_',
 "number_peaks{'n': 1}",
 '5000smoothness_entropy_',
 'ave10_7',
 'q75_roll_std_1000',
 'FFT_Mag_25q0',
 'fft_coefficientcoeff_20__attr_"abs"']
lgbm_columns = ['q25_roll_std_100',
 'abs_q25_5',
 'spkt_welch_density__coeff_3',
 'abs_q75_6',
 'abs_q75_7',
 'spkt_welch_densitycoeff_2',
 'median__roll_std',
 'abs_q01_5',
 '5000smoothness_quantile05',
 '5000smoothness_std_',
 'abs_q95_3',
 'FFT_Mag_75q0',
 '5000median_std_',
 'spkt_welch_density__coeff_17']
xgbm_columns = ['q25_roll_std_100',
 'abs_q25_5',
 'spkt_welch_density__coeff_3',
 'abs_q75_7',
 'spkt_welch_densitycoeff_2',
 "change_quantiles{'ql': 0.2, 'qh': 0.8, 'isabs': False, 'f_agg': 'var'}",
 'agg_autocorrelationf_agg_"mean"__maxlag_40',
 "number_peaks{'n': 1}",
 '5000peak_peak_amp_max_',
 'abs_q95_3',
 'spkt_welch_density__coeff_89',
 'abs_q05_2']
randomforest_randomforest = ['abs_q25_5', 'abs_q01_4', 'q25_roll_std_100']
extratrees_columns = ['q05_2',
 "number_peaks{'n': 1}",
 'abs_q01_6',
 'abs_q95_2',
 '5000smoothness_quantile25',
 '5000std_median_',
 '5000smoothness_median_',
 "change_quantiles{'ql': 0.2, 'qh': 0.8, 'isabs': False, 'f_agg': 'var'}",
 'abs_q75_6',
 "number_peaks{'n': 3}",
 'q75_roll_std_10',
 "number_peaks{'n': 10}",
 '5000min_quantile75',
 '5000smoothness_quantile05',
 "number_peaks{'n': 5}",
 'abs_q01_2',
 '5000smoothness_mean_',
 'min_roll_std_100',
 'abs_q05_2',
 'q01_roll_std_1000']
gradientboosting_columns = ['q05_5',
 'kurt_1',
 'abs_q75_6',
 'abs_q75_7',
 'spkt_welch_density__coeff_28',
 'spkt_welch_density__coeff_99',
 'fft_coefficientcoeff_6__attr_"abs"',
 '5000smoothness_quantile05',
 'q25_roll_std_100',
 'spkt_welch_densitycoeff_2',
 'abs_max_1',
 "change_quantiles{'ql': 0.2, 'qh': 0.8, 'isabs': False, 'f_agg': 'var'}",
 'abs_q25_5',
 'abs_q01_7',
 'q05_8',
 'Hilbert_mean_6',
 'abs_q95_2',
 '5000skewness_max_',
 '5000kurtosis_mean_',
 'spkt_welch_density__coeff_3']

In [10]:
all_columns = catboost_columns+lgbm_columns+xgbm_columns+randomforest_randomforest+extratrees_columns+gradientboosting_columns
unique_columns = list(set(all_columns))
common_columns = []
common_columns50 = []
common_columns75 = []
common_columns95 = []
N_columns = 6
count_values = [all_columns.count(col) for col in unique_columns]
for col in unique_columns:
    if all_columns.count(col)==N_columns:
        common_columns.append(col)
    if all_columns.count(col)>=np.quantile(count_values, .5):
        common_columns50.append(col)
    if all_columns.count(col)>=np.quantile(count_values, .75):
        common_columns75.append(col)
    if all_columns.count(col)>=np.quantile(count_values, .95):
        common_columns95.append(col)
print('unique_columns ',len(unique_columns))
print('common_columns50 ',len(common_columns50))
print('common_columns75 ',len(common_columns75))
print('common_columns95 ',len(common_columns95))
print('common_columns ',len(common_columns))

unique_columns  55
common_columns50  55
common_columns75  15
common_columns95  7
common_columns  0


In [19]:
mytrial =[]

#  tune hypterparameters
def objective(trial):
        
    max_depth = trial.suggest_int('max_depth', 3, 10)
    min_samples_split = trial.suggest_uniform('min_samples_split', .1, 1.)
    min_samples_leaf = trial.suggest_uniform('min_samples_leaf', 0, .5)
    min_weight_fraction_leaf = trial.suggest_uniform('min_weight_fraction_leaf', 0, .5)
    max_features = trial.suggest_uniform('max_features', .1, 1.)
    random_state = trial.suggest_int('random_state', 1, 9999)
        
    args={
        'columns':common_columns95,
        'kfold':{
            'n_splits': 3,
            'random_state': 1985,
            'shuffle': True,
            'type': 'group'
        },
        'scaler':{
            'cls':'StandardScaler',
            'init':{}
        },
        'algorithm':{
            'cls':'ExtraTreesRegressor',
            'init':{
                "max_depth":max_depth,
                "min_samples_split":min_samples_split,
                "min_samples_leaf":min_samples_leaf,
                "min_weight_fraction_leaf":min_weight_fraction_leaf,
                "max_features":max_features,
                "random_state":random_state,
            },
            'fit':{
            },
        },
    }
    
    df_his,  df_feature_importances, df_valid_pred, df_test_pred =  EP.process(df_train, args, df_test = df_test, trial=mytrial, remark='init tune')
    val_mae_mean = np.mean(df_his.valid)
    val_mae_var = np.var(df_his.valid)
    train_mae_mean = np.mean(df_his.train)
    train_mae_var = np.var(df_his.train)
    
    trial.set_user_attr('val_mae', val_mae_mean)
    trial.set_user_attr('train_mae', train_mae_mean)
    trial.set_user_attr('mae_diff', val_mae_mean-train_mae_mean)
    trial.set_user_attr('val_mae_var', val_mae_var)

    return np.abs(val_mae_mean - train_mae_mean)*val_mae_mean

study = optuna.create_study()
study.optimize(objective, n_trials=200)

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

[I 2019-05-25 22:22:02,795] Finished a trial resulted in value: 0.012102769938029615. Current best value is 0.012102769938029615 with parameters: {'max_depth': 9, 'min_samples_split': 0.8276790862085296, 'min_samples_leaf': 0.2567557811728629, 'min_weight_fraction_leaf': 0.04538612735529102, 'max_features': 0.5345519880452339, 'random_state': 3792}.
/home/ubuntu/anaco

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

[I 2019-05-25 22:22:03,570] Finished a trial resulted in value: 0.002678995198458539. Current best value is 0.00037247921795139483 with parameters: {'max_depth': 6, 'min_samples_split': 0.8539496795663515, 'min_samples_leaf': 0.2232982574625827, 'min_weight_fraction_leaf': 0.3978875707822334, 'max_features': 0.7427951175333791, 'random_state': 2078}.
/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

/home/ubuntu/anac

[I 2019-05-25 22:22:04,429] Finished a trial resulted in value: 0.004021666922213193. Current best value is 0.00037247921795139483 with parameters: {'max_depth': 6, 'min_samples_split': 0.8539496795663515, 'min_samples_leaf': 0.2232982574625827, 'min_weight_fraction_leaf': 0.3978875707822334, 'max_features': 0.7427951175333791, 'random_state': 2078}.
/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

[I 2019-05-25 22:

[I 2019-05-25 22:22:05,339] Finished a trial resulted in value: 0.01110620800379979. Current best value is 0.00037247921795139483 with parameters: {'max_depth': 6, 'min_samples_split': 0.8539496795663515, 'min_samples_leaf': 0.2232982574625827, 'min_weight_fraction_leaf': 0.3978875707822334, 'max_features': 0.7427951175333791, 'random_state': 2078}.
/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

[I 2019-05-25 22:2

[I 2019-05-25 22:22:06,250] Finished a trial resulted in value: 0.0021649561088931315. Current best value is 0.00037247921795139483 with parameters: {'max_depth': 6, 'min_samples_split': 0.8539496795663515, 'min_samples_leaf': 0.2232982574625827, 'min_weight_fraction_leaf': 0.3978875707822334, 'max_features': 0.7427951175333791, 'random_state': 2078}.
/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

[I 2019-05-25 22

...

[I 2019-05-25 22:22:26,623] Finished a trial resulted in value: 0.01166287488660318. Current best value is 0.00013999457391545336 with parameters: {'max_depth': 6, 'min_samples_split': 0.8905388587160279, 'min_samples_leaf': 0.10958058071163651, 'min_weight_fraction_leaf': 0.23158358513744695, 'max_features': 0.7217705744810489, 'random_state': 92}.
/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

[I 2019-05-25 22:2

[I 2019-05-25 22:22:28,255] Finished a trial resulted in value: 0.003054982329465009. Current best value is 0.00013999457391545336 with parameters: {'max_depth': 6, 'min_samples_split': 0.8905388587160279, 'min_samples_leaf': 0.10958058071163651, 'min_weight_fraction_leaf': 0.23158358513744695, 'max_features': 0.7217705744810489, 'random_state': 92}.
/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

[I 2019-05-25 22:

[I 2019-05-25 22:22:29,893] Finished a trial resulted in value: 0.0021649561088931315. Current best value is 0.00013999457391545336 with parameters: {'max_depth': 6, 'min_samples_split': 0.8905388587160279, 'min_samples_leaf': 0.10958058071163651, 'min_weight_fraction_leaf': 0.23158358513744695, 'max_features': 0.7217705744810489, 'random_state': 92}.
/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

[I 2019-05-25 22

[I 2019-05-25 22:22:31,515] Finished a trial resulted in value: 0.004522982718241731. Current best value is 6.105587831606155e-05 with parameters: {'max_depth': 6, 'min_samples_split': 0.8605564252961672, 'min_samples_leaf': 0.11044389519746498, 'min_weight_fraction_leaf': 0.48079147314816095, 'max_features': 0.8780821339313217, 'random_state': 864}.
/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

[I 2019-05-25 22:

[I 2019-05-25 22:22:33,163] Finished a trial resulted in value: 0.002482278436766747. Current best value is 6.105587831606155e-05 with parameters: {'max_depth': 6, 'min_samples_split': 0.8605564252961672, 'min_samples_leaf': 0.11044389519746498, 'min_weight_fraction_leaf': 0.48079147314816095, 'max_features': 0.8780821339313217, 'random_state': 864}.
/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

[I 2019-05-25 22:

In [20]:
for trial_i in mytrial:
    db.insert(trial_i)
df_trial = db.select()
df_trial['kfold'] = df_trial['param'].apply(lambda x: x['kfold'])

In [22]:
db.commit()

In [31]:
db.rollback()

In [35]:
mytrial = []
param = {'columns': common_columns95,
 'kfold': {'n_splits': 3,
  'random_state': 1985,
  'shuffle': True,
  'type': 'group'},
 'scaler': {'cls': 'StandardScaler', 'init': {}},
 'algorithm': {'cls': 'ExtraTreesRegressor',
  'init': {'max_depth': 6,
   'min_samples_split': 0.4652930270543402,
   'min_samples_leaf': 0.01681404462964789,
   'min_weight_fraction_leaf': 0.25844764836811196,
   'max_features': 0.9192416380489672,
   'random_state': 2236},
  'fit': {}}}

selected_columns = EP.revert_rfe(df_train, param, tsfresh_columns, df_test, mytrial, start_columns=common_columns95, limit=70, remark='start from top1 column')
print(len(selected_columns))
for trial_i in mytrial:
    db.insert(trial_i)
df_trial = db.select()
df_trial['kfold'] = df_trial['param'].apply(lambda x: x['kfold'])

In [40]:
selected_columns = ['spkt_welch_density__coeff_3',
 "change_quantiles{'ql': 0.2, 'qh': 0.8, 'isabs': False, 'f_agg': 'var'}",
 'abs_q75_6',
 'abs_q75_7',
 'abs_q25_5',
 'spkt_welch_densitycoeff_2',
 'q25_roll_std_100',
 'max_last_50000_1',
 "value_count{'value': -1}",
 'std_last_50000_9',
 'abs_max_2',
 'abs_q05_9']

In [39]:
# for trial_i in mytrial:
#     db.insert(trial_i)
# df_trial = db.select()
# df_trial['kfold'] = df_trial['param'].apply(lambda x: x['kfold'])
# df_trial[(df_trial['remark']=='start from top1 column')][['datetime','remark', 'nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff']]

In [60]:
mytrial =[]

#  tune hypterparameters
def objective(trial):
        
    max_depth = trial.suggest_int('max_depth', 3, 50)
    min_samples_split = trial.suggest_uniform('min_samples_split', .1, 1.)
    min_samples_leaf = trial.suggest_uniform('min_samples_leaf', 0, .5)
    min_weight_fraction_leaf = trial.suggest_uniform('min_weight_fraction_leaf', 0, .5)
    max_features = trial.suggest_uniform('max_features', .1, 1.)
    random_state = trial.suggest_int('random_state', 1, 9999)
        
    args={
        'columns':selected_columns,
        'kfold':{
            'n_splits': 3,
            'random_state': 1985,
            'shuffle': True,
            'type': 'stratified'
        },
        'scaler':{
            'cls':'StandardScaler',
            'init':{}
        },
        'algorithm':{
            'cls':'ExtraTreesRegressor',
            'init':{
                "max_depth":max_depth,
                "min_samples_split":min_samples_split,
                "min_samples_leaf":min_samples_leaf,
                "min_weight_fraction_leaf":min_weight_fraction_leaf,
                "max_features":max_features,
                "random_state":random_state,
            },
            'fit':{
            },
        },
    }
    
    df_his,  df_feature_importances, df_valid_pred, df_test_pred =  EP.process(df_train, args, df_test = df_test, trial=mytrial, remark='tune by stratified')
    val_mae_mean = np.mean(df_his.valid)
    val_mae_var = np.var(df_his.valid)
    train_mae_mean = np.mean(df_his.train)
    train_mae_var = np.var(df_his.train)
    
    trial.set_user_attr('val_mae', val_mae_mean)
    trial.set_user_attr('train_mae', train_mae_mean)
    trial.set_user_attr('mae_diff', val_mae_mean-train_mae_mean)
    trial.set_user_attr('val_mae_var', val_mae_var)

    return np.abs(val_mae_mean - train_mae_mean)*val_mae_mean

study = optuna.create_study()
study.optimize(objective, n_trials=200)

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

[I 2019-05-26 02:09:12,695] Finished a trial resulted in value: 2.8210982964124927e-05. Current best value is 2.8210982964124927e-05 with parameters: {'max_depth': 39, 'min_samples_split': 0.7942457471009784, 'min_samples_leaf': 0.08573747207843219, 'min_weight_fraction_leaf': 0.3299418967899597, 'max_features': 0.26548167051314386, 'random_state': 6402}.
/home/ubuntu

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

[I 2019-05-26 02:09:13,724] Finished a trial resulted in value: 0.00027299049476435294. Current best value is 2.414415104803525e-05 with parameters: {'max_depth': 21, 'min_samples_split': 0.11972897295697255, 'min_samples_leaf': 0.03019705899843539, 'min_weight_fraction_leaf': 0.00937571931451342, 'max_features': 0.9882601821602314, 'random_state': 1055}.
/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

/home/ubuntu

[I 2019-05-26 02:09:15,128] Finished a trial resulted in value: 2.1146028503537428e-05. Current best value is 2.1146028503537428e-05 with parameters: {'max_depth': 45, 'min_samples_split': 0.4156926706206989, 'min_samples_leaf': 0.05403027573305916, 'min_weight_fraction_leaf': 0.06773624308214249, 'max_features': 0.9147473731231593, 'random_state': 1318}.
/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

[I 2019-05-2

[I 2019-05-26 02:09:16,413] Finished a trial resulted in value: 0.004762284612468813. Current best value is 2.1146028503537428e-05 with parameters: {'max_depth': 45, 'min_samples_split': 0.4156926706206989, 'min_samples_leaf': 0.05403027573305916, 'min_weight_fraction_leaf': 0.06773624308214249, 'max_features': 0.9147473731231593, 'random_state': 1318}.
/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

[I 2019-05-26 

[I 2019-05-26 02:09:17,625] Finished a trial resulted in value: 0.0012582326213873773. Current best value is 2.1146028503537428e-05 with parameters: {'max_depth': 45, 'min_samples_split': 0.4156926706206989, 'min_samples_leaf': 0.05403027573305916, 'min_weight_fraction_leaf': 0.06773624308214249, 'max_features': 0.9147473731231593, 'random_state': 1318}.
/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

[I 2019-05-26

...

[I 2019-05-26 02:09:40,760] Finished a trial resulted in value: 0.0012739289781401224. Current best value is 1.913652854453704e-06 with parameters: {'max_depth': 29, 'min_samples_split': 0.318616612393517, 'min_samples_leaf': 0.1099807069388507, 'min_weight_fraction_leaf': 0.10129311035583828, 'max_features': 0.3178761317948148, 'random_state': 4120}.
/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

[I 2019-05-26 02

[I 2019-05-26 02:09:42,480] Finished a trial resulted in value: 0.0011083772305234372. Current best value is 1.913652854453704e-06 with parameters: {'max_depth': 29, 'min_samples_split': 0.318616612393517, 'min_samples_leaf': 0.1099807069388507, 'min_weight_fraction_leaf': 0.10129311035583828, 'max_features': 0.3178761317948148, 'random_state': 4120}.
/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

[I 2019-05-26 02

[I 2019-05-26 02:09:44,232] Finished a trial resulted in value: 2.8210982964124927e-05. Current best value is 1.913652854453704e-06 with parameters: {'max_depth': 29, 'min_samples_split': 0.318616612393517, 'min_samples_leaf': 0.1099807069388507, 'min_weight_fraction_leaf': 0.10129311035583828, 'max_features': 0.3178761317948148, 'random_state': 4120}.
/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

[I 2019-05-26 0

[I 2019-05-26 02:09:46,006] Finished a trial resulted in value: 0.0004677557972344763. Current best value is 1.913652854453704e-06 with parameters: {'max_depth': 29, 'min_samples_split': 0.318616612393517, 'min_samples_leaf': 0.1099807069388507, 'min_weight_fraction_leaf': 0.10129311035583828, 'max_features': 0.3178761317948148, 'random_state': 4120}.
/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

[I 2019-05-26 02

[I 2019-05-26 02:09:47,906] Finished a trial resulted in value: 0.0012376256159018033. Current best value is 1.913652854453704e-06 with parameters: {'max_depth': 29, 'min_samples_split': 0.318616612393517, 'min_samples_leaf': 0.1099807069388507, 'min_weight_fraction_leaf': 0.10129311035583828, 'max_features': 0.3178761317948148, 'random_state': 4120}.
/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

[I 2019-05-26 02

In [61]:
for trial_i in mytrial:
    db.insert(trial_i)

In [64]:
df_trial = db.select()
df_trial[(df_trial['mae_diff']<.1)].sort_values(by=['val_mae'])[['datetime','remark', 'nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff']].head(10)

,datetime,remark,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff
1211,2019-05-26 02:09:14.371371,tune by stratified,12,2.097992,0.000156,2.097786,0.000286,-0.000206
1210,2019-05-26 02:09:14.078133,tune by stratified,12,2.198667,0.000356,2.199416,0.000233,0.000749
1388,2019-05-26 02:09:46.960255,tune by stratified,12,2.241008,0.000035,2.240838,0.000084,-0.000170
1298,2019-05-26 02:09:28.585862,tune by stratified,12,2.260088,0.000024,2.259290,0.000059,-0.000798
1224,2019-05-26 02:09:16.409287,tune by stratified,12,2.311836,0.000775,2.313894,0.000582,0.002058
264,2019-05-25 22:46:50.254802,tune by group8,12,2.305445,0.014538,2.330928,0.617271,0.025483
684,2019-05-25 22:52:49.688881,tune by group3,12,2.335564,0.004635,2.331546,0.011333,-0.004018
1004,2019-05-25 22:56:26.515786,tune by group3,12,2.334600,0.002187,2.345075,0.048249,0.010475
1235,2019-05-26 02:09:18.083645,tune by stratified,12,2.353129,0.000765,2.352566,0.000724,-0.000564
1289,2019-05-26 02:09:26.900504,tune by stratified,12,2.355135,0.000005,2.355197,0.000035,0.000063


In [65]:
db.commit()

In [58]:
df_trial.shape

(1200, 14)